# Homework 2

In [1]:
import pandas as pd
import numpy as np

In [2]:
filename = "j3uwkam5ph09bjtn.csv"
df = pd.read_csv(filename)
# df.columns = ['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcd', 'at', 'ceq', 'csho', 'dlc', 'dltt', 'ebitda', 'ppent', 'costat', 'prcc_f']
df["datadate"] = pd.to_datetime(df["datadate"])
print("Dimensions:",df.shape)
df.head()

Dimensions: (580301, 17)


,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,curcd,at,ceq,csho,dlc,dltt,ebitda,ppent,costat,prcc_f
0,1000,1961-12-31,1961.0,INDL,C,D,STD,USD,NaN,NaN,0.152,NaN,0.100,NaN,NaN,I,NaN
1,1000,1962-12-31,1962.0,INDL,C,D,STD,USD,NaN,0.552,0.181,NaN,0.000,NaN,NaN,I,NaN
2,1000,1963-12-31,1963.0,INDL,C,D,STD,USD,NaN,0.553,0.186,NaN,0.015,0.046,0.431,I,NaN
3,1000,1964-12-31,1964.0,INDL,C,D,STD,USD,1.416,0.607,0.196,0.088,0.522,0.127,0.563,I,NaN
4,1000,1965-12-31,1965.0,INDL,C,D,STD,USD,2.310,0.491,0.206,0.300,1.154,-0.160,1.397,I,NaN


# Book and Market Leverage

$$\text{BookLeverage}_{i,t} = \frac{\text{DLTT}_{i,t} + \text{DLC}_{i,t}}{\text{DLTT}_{i,t} + \text{DLC}_{i,t} + \text{CEQ}_{i,t}}$$

$$\text{MarketLeverage}_{i,t} = \frac{\text{DLTT}_{i,t} + \text{DLC}_{i,t}}{\text{DLTT}_{i,t} + \text{DLC}_{i,t} + (\text{PRCC\_F}_{i,t} * \text{CSHO}_{i,t})} $$


In [3]:
df["book_leverage"] = (df["dltt"]+df["dlc"])/(df["dltt"]+df["dlc"]+df["ceq"])
df["market_leverage"] = (df["dltt"]+df["dlc"])/(df["dltt"]+df["dlc"]+df["prcc_f"]*df["csho"])

# Adding Metrics

$$\text{Assest Tangibility}_{i,t} = \frac{\text{PPENT}_{i,t}}{AT_{i,t}} $$

$$\text{Market to Book}_{i,t} = \frac{\text{DLTT}_{i,t} + \text{DLC}_{i,t} + (\text{PRCC\_F}_{i,t} * \text{CSHO}_{i,t})}{\text{DLTT}_{i,t} + \text{DLC}_{i,t} + \text{CEQ}_{i,t}} $$


In [4]:
df["asset_tangibility"] = df["ppent"]/df["at"]
df["market_to_book"] = (df["dltt"]+df["dlc"]+df["prcc_f"]*df["csho"])/(df["dltt"]+df["dlc"]+df["ceq"])
# df["profitability"] = df["ebitda"]/df["at"]
df.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,curcd,at,ceq,...,dlc,dltt,ebitda,ppent,costat,prcc_f,book_leverage,market_leverage,asset_tangibility,market_to_book
0,1000,1961-12-31,1961.0,INDL,C,D,STD,USD,NaN,NaN,...,NaN,0.100,NaN,NaN,I,NaN,NaN,NaN,NaN,NaN
1,1000,1962-12-31,1962.0,INDL,C,D,STD,USD,NaN,0.552,...,NaN,0.000,NaN,NaN,I,NaN,NaN,NaN,NaN,NaN
2,1000,1963-12-31,1963.0,INDL,C,D,STD,USD,NaN,0.553,...,NaN,0.015,0.046,0.431,I,NaN,NaN,NaN,NaN,NaN
3,1000,1964-12-31,1964.0,INDL,C,D,STD,USD,1.416,0.607,...,0.088,0.522,0.127,0.563,I,NaN,0.501233,NaN,0.397599,NaN
4,1000,1965-12-31,1965.0,INDL,C,D,STD,USD,2.310,0.491,...,0.300,1.154,-0.160,1.397,I,NaN,0.747558,NaN,0.604762,NaN


$$\text{Profitability}_{i,t} = \frac{\text{EBITDA}_{i,t}}{AT_{i,t-1}} $$


In [5]:
grouplist=[]
grouped_df = df.groupby(["gvkey"])
for gvkey, data in grouped_df:
    data["profitability"] = data["ebitda"]/data.shift(-1)["at"]
    data["asset_tangibility"] = data["asset_tangibility"].shift(-1)
    data["market_to_book"] = data["market_to_book"].shift(-1)
    grouplist.append(data)
    # tdf = pd.concat([tdf,data])
grouplist[0].head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,curcd,at,ceq,...,dltt,ebitda,ppent,costat,prcc_f,book_leverage,market_leverage,asset_tangibility,market_to_book,profitability
0,1000,1961-12-31,1961.0,INDL,C,D,STD,USD,NaN,NaN,...,0.100,NaN,NaN,I,NaN,NaN,NaN,NaN,NaN,NaN
1,1000,1962-12-31,1962.0,INDL,C,D,STD,USD,NaN,0.552,...,0.000,NaN,NaN,I,NaN,NaN,NaN,NaN,NaN,NaN
2,1000,1963-12-31,1963.0,INDL,C,D,STD,USD,NaN,0.553,...,0.015,0.046,0.431,I,NaN,NaN,NaN,0.397599,NaN,0.032486
3,1000,1964-12-31,1964.0,INDL,C,D,STD,USD,1.416,0.607,...,0.522,0.127,0.563,I,NaN,0.501233,NaN,0.604762,NaN,0.054978
4,1000,1965-12-31,1965.0,INDL,C,D,STD,USD,2.310,0.491,...,1.154,-0.160,1.397,I,NaN,0.747558,NaN,0.550617,NaN,-0.065844


In [6]:
df = pd.concat(grouplist)
df.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,curcd,at,ceq,...,dltt,ebitda,ppent,costat,prcc_f,book_leverage,market_leverage,asset_tangibility,market_to_book,profitability
0,1000,1961-12-31,1961.0,INDL,C,D,STD,USD,NaN,NaN,...,0.100,NaN,NaN,I,NaN,NaN,NaN,NaN,NaN,NaN
1,1000,1962-12-31,1962.0,INDL,C,D,STD,USD,NaN,0.552,...,0.000,NaN,NaN,I,NaN,NaN,NaN,NaN,NaN,NaN
2,1000,1963-12-31,1963.0,INDL,C,D,STD,USD,NaN,0.553,...,0.015,0.046,0.431,I,NaN,NaN,NaN,0.397599,NaN,0.032486
3,1000,1964-12-31,1964.0,INDL,C,D,STD,USD,1.416,0.607,...,0.522,0.127,0.563,I,NaN,0.501233,NaN,0.604762,NaN,0.054978
4,1000,1965-12-31,1965.0,INDL,C,D,STD,USD,2.310,0.491,...,1.154,-0.160,1.397,I,NaN,0.747558,NaN,0.550617,NaN,-0.065844


In [7]:
metric_list = ["book_leverage","market_leverage","asset_tangibility","market_to_book","profitability"]

Before Dropping NULL Values

In [8]:
df[metric_list].isnull().sum()

book_leverage        102903
market_leverage      218922
asset_tangibility    115990
market_to_book       226700
profitability        167769
dtype: int64

After dropping NULL Values

In [9]:
df.dropna(subset=metric_list, inplace=True)
df.reset_index(inplace=True, drop=True)
df[metric_list].isnull().sum()

book_leverage        0
market_leverage      0
asset_tangibility    0
market_to_book       0
profitability        0
dtype: int64

# Winsorize Outliers

In [10]:
from scipy.stats.mstats import winsorize
df['book_leverage'] = winsorize(df['book_leverage'], limits=[0.01,0.01])
df['market_leverage'] = winsorize(df['market_leverage'], limits=[0.01,0.01])

print("Dimensions:",df.shape)
df.head()

Dimensions: (287255, 22)


,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,curcd,at,ceq,...,dltt,ebitda,ppent,costat,prcc_f,book_leverage,market_leverage,asset_tangibility,market_to_book,profitability
0,1000,1970-12-31,1970.0,INDL,C,D,STD,USD,33.450,10.544,...,0.917,5.386,8.876,I,10.000,0.557700,0.352139,0.260450,1.410106,0.183635
1,1000,1971-12-31,1971.0,INDL,C,D,STD,USD,29.330,8.381,...,10.318,2.512,7.639,I,5.750,0.611199,0.433442,0.352288,1.559999,0.126187
2,1000,1972-12-31,1972.0,INDL,C,D,STD,USD,19.907,7.021,...,7.000,4.109,7.013,I,5.125,0.499251,0.320033,0.340269,0.768934,0.188737
3,1000,1973-12-31,1973.0,INDL,C,D,STD,USD,21.771,8.567,...,7.000,4.514,7.408,I,1.750,0.449669,0.584795,0.221975,0.695886,0.176067
4,1000,1974-12-31,1974.0,INDL,C,D,STD,USD,25.638,9.843,...,7.000,4.820,5.691,I,2.125,0.432451,0.621440,0.303828,0.934014,0.201631


# Fitting Regression Curves

## Book Leverage


In [11]:
from patsy import dmatrices

df = df[metric_list].copy()

y, X = dmatrices('book_leverage ~ asset_tangibility + market_to_book + profitability'
        ,data = df
        # ,return_type = "dataframe"
    )

In [17]:
df = df[np.isinf(df.market_to_book) == False]
df

,book_leverage,market_leverage,asset_tangibility,market_to_book,profitability
0,0.557700,0.352139,0.260450,1.410106,0.183635
1,0.611199,0.433442,0.352288,1.559999,0.126187
2,0.499251,0.320033,0.340269,0.768934,0.188737
3,0.449669,0.584795,0.221975,0.695886,0.176067
4,0.432451,0.621440,0.303828,0.934014,0.201631
...,...,...,...,...,...
287250,0.099198,0.120925,0.361260,1.188535,0.080999
287251,0.065895,0.055442,0.351834,1.348460,0.090943
287252,0.544077,0.719652,0.780492,0.601670,0.024741
287253,0.008831,0.004964,0.006577,2.033924,-0.156570


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [20]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
result = model.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

***
***